In [1]:
import simuvex
import angr

In [2]:
# Hacklu 2013 - reverse_me
# @author: P1kachu <p1kachu@lse.epita.fr>

In [3]:
path_types = [ 
    #'avoid',
    'errored',
    'deadended',
    'found',
    'pruned',
    'stashed',
    'unconstrained',
    'unsat'
]
def print_paths(ex, trace=False):
    for p_type in path_types:
        for path in getattr(ex, p_type):
            print("")
            print("{0}: {1}".format(p_type, path))
            if p_type == 'errored':
                print("Error: {0}".format(path.error))
            if trace:
                for step in path.trace:
                    print(step)

def i_am_clean(state):
    # Bypass ptrace/ld_peload checks
    print("I am clean at {0}".format(state.regs.eip))
    state.regs.eax = 0

def unconstrained(state):
    print("Hooked at {0}".format(state.regs.eip))

In [4]:
BINARY = 'reverse_me.bin'
fail = (0x08048e18, 0x08048711)
win  = (0x08048e16)
main = 0x080486f7
flag_addr = 0xd0000010
argv_addr = 0xd0000000

# Need something symbolic, we don't have the length !
PASS_LEN = 40

# IDA Xrefs
patch_me_rel = [0x2a, 0x2ed, 0xf7, 0x1e6, 0x3b8]
patch_me_abs = [0x0804866a, 0x080486c0]
patch_malloc = [0x66, 0x158, 0x216]
patch_strlen = [0x5e, 0xaa, 0xe2, 0x150, 0x1d1, 0x20e, 0x258, 0x36a, 0x39f]

In [5]:
p = angr.Project(BINARY)
    
p.hook(main + 0xf, func=i_am_clean, length=5)

In [6]:
init = p.factory.blank_state(addr=main)
argv=[BINARY, init.se.BVS('arg1', PASS_LEN * 8)]

init.memory.store(argv_addr, argv[0])
init.memory.store(flag_addr, argv[1])
init.regs.edi = argv_addr 
init.regs.esi = 2 # argc

In [7]:
pgp = p.factory.path_group(init)

In [8]:
ex = pgp.explore(find=win, avoid=fail)
print(ex)

I am clean at <SAO <BV32 0x8048706>>
<PathGroup with 5 avoid, 1 found, 6 errored>


In [9]:
#print_paths(ex, trace=True)
print(ex.found[0])

<Path with 1806 runs (at 0x8048e16)>


In [30]:
# tmp = ex.found[0].state.memory.load(argv_addr, 40 * 8)
# print(tmp)
s = ex.found[0].state
print(s.simplify())
for x in xrange(PASS_LEN):
    x = ex.found[0].state.memory.load(flag_addr + x, 1)
    if x.args[0] in xrange(0, 256):
        print(chr(x.args[0]))

None
�
�
�
�
�
�
�
�
�
�
�
�
�
�
�
�

w
o
g
_
W
O
G
?
7
/
'




